In [ ]:
import pandas as pd

def handler(context, event):
    # Load images from local paths via CSV
    # Perform image transformations and splits 
    # Output data loaders (train, validation, test)
    
    context.logger.info("Reading CSV")
    df = pd.read_csv(str(context.inputs['local_images_csv']))
    print(df)